In [2]:
# !pip install scikit-learn
# !pip  install tensorflow
# !pip install pandas

In [3]:
import pandas as pd 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
iris = load_iris()

# Create a DataFrame for the features
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
# df['target'] = iris.target
print(df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2


In [12]:
targets = pd.DataFrame()
targets['target'] = iris.target

X_train, X_test, Y_train, Y_test = train_test_split(df, targets, test_size=0.2, random_state = 40)

In [13]:
X_train.shape

(120, 4)

In [14]:
class MLP_Neural_Network:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5, 
                                                        (self.input_nodes, self.hidden_nodes))
        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5,
                                                         (self.hidden_nodes, self.output_nodes))

        self.lr = learning_rate
        self.activation_function = lambda x: 1/(1+np.exp(-x))

    def forward(self, X):
        hidden_inputs = np.dot(X, self.weights_input_to_hidden)
        hidden_outputs = self.activation_function(hidden_inputs)

        final_inputs = np.dot(hidden_outputs, self.weights_hidden_to_output)
        final_outputs = self.activation_function(final_inputs) 

        return hidden_outputs, final_outputs
    
    def backward_prop(self, hidden_outputs, final_outputs, X, Y, delta_weights_i_h, delta_weights_h_o):
        error = Y - final_outputs
        hidden_error = np.dot(self.weights_hidden_to_output,error)
        output_error_term = error * final_outputs * (1-final_outputs)
        hidden_error_term = hidden_error * hidden_outputs * (1-hidden_outputs)

        delta_weights_h_o += output_error_term*hidden_outputs[:,None]
        delta_weights_i_h += hidden_error_term*X[:,None]

        return delta_weights_i_h, delta_weights_h_o
    
    def update_weights(self, delta_weights_i_h, delta_weights_h_o, n_records):
        self.weights_input_to_hidden+=self.lr*delta_weights_i_h/n_records
        self.weights_hidden_to_output+=self.lr*delta_weights_h_o/n_records

    def train(self, features, targets):
        n_records = features.shape[0]
        delta_weights_i_h = np.zeros(self.weights_input_to_hidden.shape)
        delta_weights_h_o = np.zeros(self.weights_hidden_to_output.shape)

        for X,Y in zip(features,targets):
            hidden_output, final_output = self.forward(X)
            delta_weights_i_h, delta_weights_h_o = self.backward_prop(hidden_output,
                                                                      final_output,
                                                                      X,
                                                                      Y,
                                                                      delta_weights_i_h,
                                                                      delta_weights_h_o)
        self.update_weights(delta_weights_i_h, delta_weights_h_o, n_records)

    def predict(self, features):
        hidden_inputs = np.dot(features, self.weights_input_to_hidden)
        hidden_outputs = self.activation_function(hidden_inputs)

        final_inputs = np.dot(hidden_outputs, self.weights_hidden_to_output)
        final_outputs = final_inputs

        return final_outputs


In [15]:
iterations=100
learning_rate=0.01
hidden_nodes=2
output_nodes=1

In [16]:
def MSE(y,Y):
  return np.mean((y-Y)**2)

In [17]:
import unittest

inputs = np.array([[0.5, -0.2, 0.1]])
targets = np.array([[0.4]])

test_w_i_h = np.array([[0.1, -0.2],
                       [0.4, 0.5],
                       [-0.3, 0.2]])
test_w_h_o = np.array([[0.3],
                       [-0.1]])

class TestMethods(unittest.TestCase):
        
    def test_data_loaded(self):
        # Test that data frame loaded
        self.assertTrue(isinstance(df, pd.DataFrame))

    def test_activation(self):
        network = MLP_Neural_Network(3, 2, 1, 0.5)
        # Test that the activation function is a sigmoid
        self.assertTrue(np.all(network.activation_function(0.5) == 1/(1+np.exp(-0.5))))

    def test_train(self):
        # Test that weights are updated correctly on training
        network = MLP_Neural_Network(3, 2, 1, 0.5)
        network.weights_input_to_hidden = test_w_i_h.copy()
        network.weights_hidden_to_output = test_w_h_o.copy()
        
        network.train(inputs, targets)
        print(network.weights_hidden_to_output)
        self.assertTrue(np.allclose(network.weights_hidden_to_output, 
                                    np.array([[ 0.29244214], 
                                              [-0.10709219]])))
        self.assertTrue(np.allclose(network.weights_input_to_hidden,
                                    np.array([[ 0.10562014, -0.20185996], 
                                              [0.39775194, 0.50074398], 
                                              [-0.29887597, 0.19962801]]),rtol=1.0))
    def test_run(self):
        # Test correctness of run method
        network = MLP_Neural_Network(3, 2, 1, 0.5)
        network.weights_input_to_hidden = test_w_i_h.copy()
        network.weights_hidden_to_output = test_w_h_o.copy()

        self.assertTrue(np.allclose(network.predict(inputs), 0.09998924))

suite = unittest.TestLoader().loadTestsFromModule(TestMethods())
unittest.TextTestRunner().run(suite)


....

[[ 0.29244214]
 [-0.10709219]]



----------------------------------------------------------------------
Ran 4 tests in 0.021s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [21]:
import sys
N_i=X_train.shape[1]
network=MLP_Neural_Network(N_i,hidden_nodes,output_nodes,learning_rate)

losses={'train':[], 'validation':[]}
for ii in range(3):
    batch=np.random.choice(X_train.index,size=32)
    X,y=X_train.iloc[batch].values, Y_train.iloc[batch]
    network.train(X,y)

    train_loss = MSE(network.run(X_train).T, Y_train['target'].values)
    val_loss = MSE(network.run(X_test).T, Y_test['target'].values)
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations)) \
                    + "% ... Training loss: " + str(train_loss)[:5] \
                    + " ... Validation loss: " + str(val_loss)[:5])
    sys.stdout.flush()
    
    losses['train'].append(train_loss)
    losses['validation'].append(val_loss)

[46 28 48 48 17  5 85 49 71 61 69 22 19  8 23 30 67 37 70 49 72 83 17  0
 47 40 60 59 30 21 84 27]


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U6'), dtype('float64')) -> None